In [1]:
#!pip install tf-nightly
#! conda install -c conda-forge ffmpeg
#! pip install opencv-python
# pip install opencv-contrib-python
# pip install pynput


In [11]:
# Autor: Lautaro Linquiman - lylinquiman@gmail.com

import cv2 as cv
import numpy as np
import statistics as stats

from pynput.keyboard import Key, Controller
from queue import Queue
%matplotlib inline
from matplotlib import pyplot as plt

import time
import numpy as np
import math
import threading

In [2]:
# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

# Librerias de ayuda
import numpy as np

print(tf.__version__)

2.3.0


In [3]:
from main import ceil_deteccion, obstacle_deteccion, ball_deteccion, move_left, move_right

In [4]:
model = tf.keras.models.load_model('direccion_model')
speed_model = tf.keras.models.load_model('speed_model')

In [27]:
import scipy.stats
#fig = plt.figure()
#ax = fig.gca()
#fig.show()
resize = (64, 48)
img_samples = []

def processing(img):
    global img_samples
    #print(img)
   
    img = cv.cvtColor(img, cv.COLOR_RGB2HSV) 
    ceil = ceil_deteccion(img)
    ceil[ceil > 0] = 255
    obs = obstacle_deteccion(img)
    ball = ball_deteccion(img, None)[2]
    ball[ball > 0] = 50

    all_ = ceil + obs + ball
    #print(all_.shape)
    img = cv.resize(all_, resize)
    #img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    #print(img.shape)
    #img_end = np.array(img_end)
    
    #img = cv.cvtColor(img, cv.COLOR_RGB2GRAY) 
    #print(img.shape)
    img = np.array(img)
    img = img.astype('float32')
    img /= 255 
    #print(img.shape)
    #img = tf.reshape(img, [64, 48])
    #print(img)
    #cv.imshow("test", img)
    #cv.waitKey(0)
    #cv.destroyAllWindows()
    
    #ax.set_title("00")
    #fig.canvas.draw()
    #plt.imshow(img)
    #plt.show()
    img_samples.append(img)
    if len(img_samples) > 3: 
        predictions = model.predict(np.array(img_samples))
        speed_predictions = speed_model.predict(np.array(img_samples)).flatten()
        #print(predictions)
        #predictions = scipy.stats.mode(predictions)
        predictions = np.average(predictions, axis=1)
        #print(speed_predictions)
        speed_predictions = (np.average(speed_predictions)/1000)
        #print(speed_predictions)
        #test_loss, test_acc = model.evaluate(img,  np.array(np.argmax(predictions)), verbose=2)

        #print('\nTest accuracy:', test_acc)
        #img_samples = []
        img_samples = []
        return (np.argmax(predictions), speed_predictions)
    return (3, 3)

In [28]:
from pynput.keyboard import Key, Controller
keyboard = Controller()
import time
#last = 0
def move(q):
    last = 0
    while True:
        direccion,d = q.get()
        
        #if last == direccion:
        #    continue
        if direccion == 0:
            keyboard.release(Key.right)        
            keyboard.release(Key.left)
            time.sleep(d/5)
        elif direccion == 1:  
            keyboard.release(Key.right)
            keyboard.press(Key.left)
            #time.sleep(d)
            
            #keyboard.release(Key.left)
        elif direccion == 2: 
            keyboard.release(Key.left)
            keyboard.press(Key.right)
            #time.sleep(d)
            #keyboard.release(Key.right)
        last = direccion
        #time.sleep(0.05)
    

In [29]:
import IPython

In [30]:
cap = cv.VideoCapture(0)

#640.0 x 480.0
cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
import threading
import queue
q = queue.Queue()
q.put([3,0])
t = threading.Thread(target=(move), args=(q,))
t.start()
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:  # if ret is True:
        IPython.display.clear_output(wait=True)
        #frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        #print(ret)
        #frame = cv.rotate(frame, cv.ROTATE_180)
        frame = cv.flip(frame,-1)
        frame = cv.flip(frame,1)
        #plt.figure()
         # Turn off the axis
        #plt.axis('off')
        # Title of the window
        #plt.title("Input Stream")
        # Display the frame
        #plt.imshow(frame)
        #plt.show()
        #plt.figure()
        #plt.imshow(frame)
        #plt.show()
        #cv.imshow("",frame)
        # Display the frame until new frame is available
        
        #plt.show()
        r,s = processing(frame)
        if r == False:
            #print("False")
            pass
        #if r == 0:
        #    move_left(0.1)
        #elif r == 1:
        #    move_right(0.1)
        print("{} - {}".format(r, s))
        if r < 3:
            
            q.queue.clear()
            q.put([r, s])
      # ... other code ...
        
    else:
          print("empty frame")      


3 - 3


KeyboardInterrupt: 